In [11]:
# ------ 1. 데이터 전처리 ------

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 12


In [12]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [17]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [18]:
# 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터
# 맨 우측에 있는 단어만 레이블로 분리
print(sequences)


[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [19]:
# 전체 샘플에 대해 길이를 맞춰준다. 가장 긴 샘플의 길이를 기준으로.
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이: ', max_len)

샘플의 최대 길이:  6


In [21]:
# 패딩 진행
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)


[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [23]:
# 마지막 단어만 레이블로 분리.
# numpy 사용
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X)
print(y)


[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [24]:
# RNN 모델에 훈련시키기 위해 레이블을 원-핫 인코딩 수행
y = to_categorical(y)
print(y)


[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [25]:
# ------ 2. 모델 설계하기 ------
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)


Epoch 1/200
1/1 - 0s - loss: 2.4923 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4803 - accuracy: 0.0909
Epoch 3/200
1/1 - 0s - loss: 2.4688 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4574 - accuracy: 0.0909
Epoch 5/200
1/1 - 0s - loss: 2.4461 - accuracy: 0.1818
Epoch 6/200
1/1 - 0s - loss: 2.4347 - accuracy: 0.2727
Epoch 7/200
1/1 - 0s - loss: 2.4231 - accuracy: 0.3636
Epoch 8/200
1/1 - 0s - loss: 2.4112 - accuracy: 0.3636
Epoch 9/200
1/1 - 0s - loss: 2.3988 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3859 - accuracy: 0.3636
Epoch 11/200
1/1 - 0s - loss: 2.3725 - accuracy: 0.3636
Epoch 12/200
1/1 - 0s - loss: 2.3584 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.3436 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.3281 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.3118 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2946 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2765 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2575 - accuracy: 0.3636
E

Epoch 147/200
1/1 - 0s - loss: 0.2464 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2413 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2363 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.2315 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.2267 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.2221 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.2176 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.2133 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.2090 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.2048 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.2007 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1968 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1929 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1891 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1854 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1818 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1783 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [31]:
model.predict_classes([10, 1])

array([1, 4], dtype=int64)

In [48]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        print(encoded)
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            if index == result:
                current_word = current_word + ' ' + word
                break
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [49]:
print(sentence_generation(model, t, '그의', 2))

[[0 0 0 0 6]]
[[0 0 0 6 1]]
그의 말이 법이다


In [52]:
print(sentence_generation(model, t, '경마장에', 4))

[[0 0 0 0 2]]
[[0 0 0 2 3]]
[[0 0 2 3 1]]
[[0 2 3 1 4]]
경마장에 있는 말이 뛰고 있다
